# Sample Conversation

In [5]:
import json
train_conversations_path = './data/train_set/train_set_ALL.json'

# load training set
with open(train_conversations_path, "r") as data:
    conversations = json.load(data)
print("%d conversations loaded"%len(conversations))

6720 conversations loaded


In [9]:
# load a sample conversation as a sequence of entities
conversation = conversations[0]
seed_entity = conversation['seed_entity']
print(seed_entity)

dialogue_history = []
for i in range(len(conversation['questions'])):
    answer = conversation['questions'][i]['answer']
    # consider only answers which are entities
    if ('www.wikidata.org' in answer):
        print(answer)

# TODO retrieve relations between these entities

https://www.wikidata.org/wiki/Q223495
https://www.wikidata.org/wiki/Q928282
https://www.wikidata.org/wiki/Q131749
https://www.wikidata.org/wiki/Q181754


# Retrieve Relations

In [ ]:
# load 